#Edge Detection

##環境(Environment)
* 作業環境(notebook)&nbsp;: Google Colaboratory<br>
* 言語&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;(language) : Python2

##Data Set

[SK-LARGE](http://kaizhao.net/sk-large)

This dataset, namely SK-LARGE, is used for evaluating the methods for skeleton extraction in natural images, which contents 1491 images selected from MS COCO [1], 746 for training and 745 for testing.

※学習済みモデルを使用しているため、学習データの紹介のみします。

In [0]:
# download and extract data
!wget http://data.kaizhao.net/projects/skeleton/sk1491.tar.gz
!tar -xvf sk1491.tar.gz

In [0]:
# data augmentation
!wget http://data.kaizhao.net/projects/skeleton/augmentation.m
!matlab -nodisplay -r "run augmentation.m; exit"

##Model
###[VGG16](http://caffe.berkeleyvision.org/model_zoo.html)
* モデル形式：caffe model

<br/>
<br/>
We recommand you execute below codes because of compiling <br>
以下はColab環境でコンパイル動作を行うための必要な処理です。


In [0]:
!git init
!git pull https://github.com/zeakey/skeleton

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 452, done.
remote: Total 452 (delta 0), reused 0 (delta 0), pack-reused 452
Receiving objects: 100% (452/452), 1.25 MiB | 2.57 MiB/s, done.
Resolving deltas: 100% (111/111), done.
From https://github.com/zeakey/skeleton
 * branch            HEAD       -> FETCH_HEAD
error: The following untracked working tree files would be overwritten by merge:
	models/fsds_test.prototxt
Please move or remove them before you merge.
Aborting


In [0]:
# model copy
!ln -s "/content/drive/My Drive/Colab Notebooks/skeleton/vgg16convs.caffemodel" "/content/models/vgg16convs.caffemodel"

In [0]:
!apt install -y caffe-cuda

In [0]:
!sudo apt-get install libgflags-dev libgoogle-glog-dev liblmdb-dev

In [0]:
!sudo apt-get install libprotobuf-dev protobuf-compiler

In [0]:
!sudo apt-get install libleveldb-dev

In [0]:
!sudo apt-get install -y libsnappy-dev

In [0]:
!pip install cmake --upgrade
!cmake -D USE_CUDNN=OFF caffe/

In [0]:
!make pycaffe

In [0]:
!sudo apt-get install octave-image

In [0]:
%cd /content
!octave -x --eval  "pkg install -forge liboctave-dev image"
!echo "pkg load image;" >> .octaverc

In [0]:
!octave --no-gui -x augmentation.m

In [0]:
# ! git clone https://github.com/BVLC/caffe.git
!cp caffe/Makefile.config.example caffe/Makefile.config

In [0]:
!make -f /content/caffe/Makefile




In [0]:
!ln -s "/content/drive/My Drive/Colab Notebooks/fsds-pretrained-skl.zip" "/content/fsds-pretrained-skl.zip"

In [0]:
import zipfile
with zipfile.ZipFile('/content/fsds-pretrained-skl.zip') as existing_zip:
    existing_zip.extractall('/content/models')

##実行
###以下の写真はセグメンテーションの処理済の写真です。
   <img src="https://user-images.githubusercontent.com/39483767/84374705-59dc6f80-ac19-11ea-9ff6-fe611a2db7b0.jpg" width="40%"> </img>
<br>

###この写真を準備されたソースコードより処理すると以下の写真になりあす.

   <img src="https://user-images.githubusercontent.com/39483767/84405495-2c0c2080-ac43-11ea-872e-ead07af4ffc4.jpg" width="40%"> </img>


<br>
<font size="2">※現在の時点ではセグメンテーションモデルの学習データセットがないため、手作業で行った結果です。データセットはいま申し込んで待っている状態です。</font> 


In [0]:
!python2 forward_all.py

I0331 11:38:47.617511  4944 net.cpp:50] Initializing net from parameters: 
name: "FSDS_TEST"
input: "data"
input_dim: 1
input_dim: 3
input_dim: 200
input_dim: 200
state {
  phase: TEST
}
layer {
  name: "conv1_1"
  type: "Convolution"
  bottom: "data"
  top: "conv1_1"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 64
    pad: 35
    kernel_size: 3
    engine: CAFFE
  }
}
layer {
  name: "relu1_1"
  type: "ReLU"
  bottom: "conv1_1"
  top: "conv1_1"
}
layer {
  name: "conv1_2"
  type: "Convolution"
  bottom: "conv1_1"
  top: "conv1_2"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 64
    pad: 1
    kernel_size: 3
    engine: CAFFE
  }
}
layer {
  name: "relu1_2"
  type: "ReLU"
  bottom: "conv1_2"
  top: "conv1_2"
}
layer {
  name: "pool1"
  type: "Pooling"
  bottom: "conv1_2"
  top: "pool1"
  pooling_param {
    pool: MA

##後処理
skeletonizationされた写真より今後のデータ処理のため,収縮(Erosion)と膨張(Dilation)を用いたモルフォロジー変換を行います。

In [0]:
import cv2
import numpy as np
 
img = cv2.imread('/content/test/abc.jpg',0)
size = np.size(img)
skel = np.zeros(img.shape,np.uint8)
 
ret,img = cv2.threshold(img,127,255,0)
element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
done = False
 
while( not done):
    eroded = cv2.erode(img,element)
    temp = cv2.dilate(eroded,element)
    temp = cv2.subtract(img,temp)
    skel = cv2.bitwise_or(skel,temp)
    img = eroded.copy()
 
    zeros = size - cv2.countNonZero(img)
    if zeros==size:
        done = True
 
cv2.imwrite("/content/test/abc_skel.png",skel)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [0]:
import cv2
img = cv2.imread('/content/data/sk-results/fsds-skl-pretrained/abc_skel.png',0)
element = cv2.getStructuringElement(cv2.MORPH_CROSS,(2,2))

dst = cv2.dilate(img,element) # 膨張処理

dst = cv2.erode(dst,element) # 収縮処理
cv2.imwrite("ero_dil2.png",dst )

True

##Result  
   <span>
   <img src="https://user-images.githubusercontent.com/39483767/84374677-4f21da80-ac19-11ea-9aba-ff9419018e1e.png" width="40%"> </img>
   </span>